In [1]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip install llama-index llama-index-llms-azure-openai llama-index-embeddings-azure-openai chromadb llama-index-vector-stores-chroma
!pip install openai-whisper pdf2image pytesseract
!apt-get update -qq
!apt-get install -y -qq poppler-utils

  Using cached openai_whisper-20250625.tar.gz (803 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=4f7d531a5a4f75e80748c4a3f1945821a249f3aee66f52420316973a26d2a0d0
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [openai-whisper]
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11)

In [3]:
from google.colab import userdata

AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [4]:
from llama_index.llms.azure_openai import AzureOpenAI

# ====== Step 1: Initialize the LLM ======
llm = AzureOpenAI(
    model=DEPLOYMENT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2023-05-15",
)

In [5]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ====== Step 2: Initialize Azure OpenAI embedding model ======
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",  # model type
    deployment_name="text-embedding-3-small",
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2023-05-15",
)

In [6]:
from llama_index.core import Settings

# ====== STEP 3: Register models globally (important!) ======
Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
# setup a client

import chromadb
client = chromadb.Client()

In [8]:
# If you want to use Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')

# Or upload single files manually:
from google.colab import files
files.upload()  # uncomment to open upload dialog


Saving call-1.mp3 to call-1.mp3
Saving invoice-1234.pdf to invoice-1234.pdf


{'call-1.mp3': b'ID3\x03\x00\x00\x00\x00\x03%TFLT\x00\x00\x00\x0f\x00\x00\x01\xff\xfeM\x00P\x00G\x00/\x003\x00\x00\x00COMM\x00\x00\x00\x82\x00\x00\x00engiTunSMPB\x00 00000000 00000000 00000000 000000000011c48f 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [9]:
# ==========================
# Robust PDF + Audio -> Documents
# ==========================

import os
from pdf2image import convert_from_path
import pytesseract
from llama_index.core import Document
import whisper

# --------------------------
# Paths (change as needed)
# --------------------------
PDF_PATH = "/content/data/invoice-1234.pdf"   # PDF file path
AUDIO_PATH = "/content/data/call-1.mp3"   # Audio file path
OUT_DIR = "/content/data/pdf_pages"               # folder to store page images
os.makedirs(OUT_DIR, exist_ok=True)

# --------------------------
# 1) OCR PDF pages
# --------------------------
pdf_filename = os.path.basename(PDF_PATH)
texts = []

try:
    pages = convert_from_path(PDF_PATH, dpi=300, fmt='jpeg', output_folder=OUT_DIR)
except Exception as e:
    raise RuntimeError("Error reading PDF. Is poppler installed?") from e

for i, page in enumerate(pages):
    img_path = os.path.join(OUT_DIR, f"page_{i+1}.jpg")
    page.save(img_path, "JPEG")
    text = pytesseract.image_to_string(img_path)
    texts.append({
        "page": i + 1,
        "text": text or "",
        "source": pdf_filename
    })

print(f"OCR done. Pages processed: {len(texts)}")

# --------------------------
# 2) Transcribe Audio (Whisper)
# --------------------------
# Make sure correct package installed: pip install openai-whisper
model = whisper.load_model("small")  # can use "base" or "medium"
result = model.transcribe(AUDIO_PATH, language="en")
audio_text = result.get("text", "")

print(f"Audio transcription length: {len(audio_text)} characters")

# --------------------------
# 3) Create Documents for LlamaIndex
# --------------------------
documents = []

# PDF documents
for item in texts:
    doc = Document(
        text=item.get("text", ""),
        extra_info={
            "source": item.get("source", "unknown"),
            "page": item.get("page", None)
        }
    )
    documents.append(doc)

# Audio document
doc_audio = Document(
    text=audio_text,
    extra_info={
        "source": os.path.basename(AUDIO_PATH),
        "type": "audio"
    }
)
documents.append(doc_audio)

print(f"Total documents created: {len(documents)}")


OCR done. Pages processed: 1


100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 56.3MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Audio transcription length: 297 characters
Total documents created: 2


In [10]:
persist_directory = "db"

In [11]:
# ==========================
# RAG ingestion: Chroma + Azure OpenAI
# ==========================

import os
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from chromadb import PersistentClient

# --------------------------
# Paths / persistence
# --------------------------
persist_directory = "/content/chroma_db"
os.makedirs(persist_directory, exist_ok=True)



# --------------------------
# 2) Setup persistent Chroma client
# --------------------------
chroma_client = PersistentClient(path=persist_directory)
chroma_collection = chroma_client.get_or_create_collection("llamaindex_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# --------------------------
# 3) Build index with your documents
# --------------------------
# 'documents' is your list of Document objects from PDF/audio ingestion
index = VectorStoreIndex(
    documents,
    storage_context=storage_context,
    embed_model=embed_model,
)

# --------------------------
# 4) Persist to disk
# --------------------------
index.storage_context.persist(persist_dir=persist_directory)
print(f"Index created and persisted with {len(documents)} documents.")


Index created and persisted with 2 documents.


In [12]:
retriever = index.as_retriever(similarity_top_k=4)

# Inspect retriever configuration
print(retriever.similarity_top_k)

4


In [13]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=llm,
    response_mode="compact"  # or "tree_summarize", "refine", etc.
)

In [14]:
response = query_engine.query("Summarize the pdf")
print(response.response)  # answer text
print(response.source_nodes)  # retrieved sources


The document is an invoice from Contoso Ltd, dated 03/07/2025, addressed to John Smith at 123 River Street, Marshtown, England. It lists purchased items including a 38mm Widget, a pack of 3.5mm screws, and a left-handed screwdriver, with their respective prices, quantities, and item totals. The subtotal is £61.48, with £6.14 tax and £15.00 shipping, resulting in a total amount due of £82.62.
[NodeWithScore(node=TextNode(id_='e34dc139-e67c-4f87-b961-b9056028afcc', embedding=None, metadata={'source': 'invoice-1234.pdf', 'page': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Contoso Ltd Invoice No: 1234\n2 Main St, Bigtown, England, EH1 234\nTel: 555 123-4567 Date: 03/07/2025\n\nCustomer Name: John Smith\n\nAddress: 123 River Street\nMarshtown\nEngland\nGL1 234\nItem Price Quantity Item Total\n38mm Widget 24.50 2 49.00\n3.5mm screws pack 4.99 1 4.99\nLeft-handed screwdriver 7.49 1 7.4

In [15]:
response = query_engine.query("Summarize the mp3")
print(response.response)  # answer text
print(response.source_nodes)  # retrieved sources

Ava from Contoso called to follow up on a meeting from the previous week. She mentioned that she has reviewed the numbers and believes they can meet the price expectations. She requested a callback at 555-12345 or an email at ava.contoso.com to discuss the next steps.
[NodeWithScore(node=TextNode(id_='38e768a3-4702-4dcc-8b02-97d7d4defa60', embedding=None, metadata={'source': 'call-1.mp3', 'type': 'audio'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text=" Hi, this is Ava from Contoso. Just calling to follow up on our meeting last week. I wanted to let you know that I've run the numbers and I think we can meet your price expectations. Please call me back on 555-12345 or send me an email at ava.contoso.com and we'll discuss next steps. Thanks. Bye.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0

In [16]:
response = query_engine.query("what ava was taking")
print(response.response)  # answer text
print(response.source_nodes)  # retrieved sources

Ava was discussing a follow-up on a meeting and mentioned that she had reviewed the numbers and believed they could meet the price expectations. She requested a callback or an email to discuss the next steps.
[NodeWithScore(node=TextNode(id_='38e768a3-4702-4dcc-8b02-97d7d4defa60', embedding=None, metadata={'source': 'call-1.mp3', 'type': 'audio'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text=" Hi, this is Ava from Contoso. Just calling to follow up on our meeting last week. I wanted to let you know that I've run the numbers and I think we can meet your price expectations. Please call me back on 555-12345 or send me an email at ava.contoso.com and we'll discuss next steps. Thanks. Bye.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.3050310179732278), NodeWithScore(node=TextNode(id_='e34dc13

In [17]:
response = query_engine.query("what john smith has purchased")
print(response.response)  # answer text
print(response.source_nodes)  # retrieved sources

John Smith has purchased the following items:  
- Two 38mm Widgets for a total of £49.00  
- One pack of 3.5mm screws for £4.99  
- One left-handed screwdriver for £7.49
[NodeWithScore(node=TextNode(id_='e34dc139-e67c-4f87-b961-b9056028afcc', embedding=None, metadata={'source': 'invoice-1234.pdf', 'page': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Contoso Ltd Invoice No: 1234\n2 Main St, Bigtown, England, EH1 234\nTel: 555 123-4567 Date: 03/07/2025\n\nCustomer Name: John Smith\n\nAddress: 123 River Street\nMarshtown\nEngland\nGL1 234\nItem Price Quantity Item Total\n38mm Widget 24.50 2 49.00\n3.5mm screws pack 4.99 1 4.99\nLeft-handed screwdriver 7.49 1 7.49\nSubtotal 61.48\nTax 6.14\nShipping 15.00\n\nTotal Due 82.62\n\x0c', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.333469491531